In [33]:
import time
import sys
import os
import re
import pandas as pd
import numpy as np
import getpass
from datetime import datetime, timedelta

In [19]:
import warnings
warnings.filterwarnings("ignore")

In [64]:
last_week_start = datetime.strftime(datetime.now() - timedelta(days=datetime.now().weekday() + 7),"%Y%m%d")
last_week_end = datetime.strftime(datetime.now() - timedelta(days=datetime.now().weekday() + 1),"%m%d")

In [65]:
print(last_week_start,last_week_end)

20210503 0509


In [68]:
account=getpass.getuser()

#注意windows用户请把'/Users/'改为 ’c:/Users/‘
path='/Users/'+str(account)+'/Desktop/weekly_kw_update/'

#这里需要修改rawdata的文件夹名字，每更新一周就改一个名字，这里对齐就好
raw_path=path+"week_"+str(last_week_start)+"-"+str(last_week_end)+"/"
print(raw_path)

/Users/godmax/Desktop/weekly_kw_update/week_20210503-0509/


In [21]:
#需要提前把dictionary文件夹放入weekly_kw_update这个文件夹里
cat_dict0=pd.read_excel(path+'dictionary.xlsx',sheet_name='cat_dict')
brand_dict0=pd.read_excel(path+'dictionary.xlsx',sheet_name='brand_dict')

In [ ]:
cat_dict=dict(zip(cat_dict0["subcat"],cat_dict0["cat"]))
brand_dict=dict(zip(brand_dict0["sub_kw"],brand_dict0["brand"]))

In [22]:
cat_kw=dict()
cat_decrp=dict()
brand_kw=dict()
brand_descrp=dict()
for file in os.listdir(raw_path):
    buff=file.split(sep="_")
    df=pd.read_csv(raw_path+file)
    
    if buff[1].lower() in [x.lower() for x in cat_dict.keys()]:
        if "相关搜索" in buff[5]:
            print(buff[1])
            cat_kw[buff[1]]=df
        if "关联修饰" in buff[5]:
            cat_decrp[buff[1]]=df
    if buff[1].lower() in [x.lower() for x in brand_dict.keys()]:
        if "相关搜索" in buff[5]:
            brand_kw[buff[1]]=df
        if "关联修饰" in buff[5]:
            brand_descrp[buff[1]]=df
            

In [23]:
cat_kw2=pd.concat(cat_kw,axis=0).reset_index()
cat_decrp2=pd.concat(cat_decrp,axis=0).reset_index()
brand_kw2=pd.concat(brand_kw,axis=0).reset_index()
brand_descrp2=pd.concat(brand_descrp,axis=0).reset_index()


马丁靴
女包
运动t恤
运动服
卡套
羽绒服
手机包
卡包
双肩包
帆布鞋
徒步鞋
女装
运动卫衣
板鞋
夹克
皮肤衣
休闲裤
运动鞋
冲锋衣
卫衣
靴子
运动外套
钥匙包
休闲鞋
t恤
证件包
滑雪服


In [ ]:
def add_yw(data,year,week_num):
    data=data.loc[data["日期"].isna()==False]
    data["Year"]=year
    data["Week"]=week_num
    data["Week_Detail"]=data["日期"]
    data2=data[["Year","Week","Week_Detail"]+[x for x in data.columns if x not in ["level_1","日期","Year","Week","Week_Detail"]]]
    return(data2)

In [24]:
week_num=18

cat_kw3=add_yw(cat_kw2,2021,week_num)
cat_kw3=cat_kw3.rename(columns={"level_0":"keywords"},inplace=False)
cat_kw3["category"]=[cat_dict[x.lower()] for x in cat_kw3["keywords"]]
print(cat_kw3.head(3))

cat_decrp3=add_yw(cat_decrp2,2021,week_num)
cat_decrp3=cat_decrp3.rename(columns={"level_0":"keywords"},inplace=False)
cat_decrp3["category"]=[cat_dict[x.lower()] for x in cat_decrp3["keywords"]]
print(cat_decrp3.head(3))

brand_kw3=add_yw(brand_kw2,2021,week_num)
brand_kw3=brand_kw3.rename(columns={"level_0":"Search_keywords"},inplace=False)
brand_kw3["Brand"]=[brand_dict[x.lower()] for x in brand_kw3["Search_keywords"]]
print(brand_kw3.head(3))

brand_descrp3=add_yw(brand_descrp2,2021,week_num)
brand_descrp3=brand_descrp3.rename(columns={"level_0":"Search_keywords"},inplace=False)
brand_descrp3["Brand"]=[brand_dict[x.lower()] for x in brand_descrp3["Search_keywords"]]
print(brand_descrp3.head(3))

In [ ]:
# 获取ExcelWriter对象
target_path=path+"关键词&修饰词_weekly_till_"+str(last_week_end)+"阿明.xlsx"
pd.DataFrame().to_excel(target_path)
writer = pd.ExcelWriter(target_path,engine='xlsxwriter',options={'strings_to_numbers': True}) 
 
#将df1与df2写入writer中
brand_kw3[["Year","Week","Week_Detail","Brand"]+[x for x in brand_kw3.columns if x not in ["Year","Week","Week_Detail","Brand"]]].to_excel(writer, sheet_name='品牌关键词',index=False)
brand_descrp3[["Year","Week","Week_Detail","Brand"]+[x for x in brand_descrp3.columns if x not in ["Year","Week","Week_Detail","Brand"]]].to_excel(writer, sheet_name='品牌修饰词',index=False)
cat_kw3[["Year","Week","Week_Detail","category"]+[x for x in cat_kw3.columns if x not in ["Year","Week","Week_Detail","category"]]].to_excel(writer, sheet_name='Category关键词',index=False)
cat_decrp3[["Year","Week","Week_Detail","category"]+[x for x in cat_decrp3.columns if x not in ["Year","Week","Week_Detail","category"]]].to_excel(writer, sheet_name='Category修饰词',index=False)
 
#保存writer
writer.save() 
writer.close()